In [12]:
from dotenv import load_dotenv
import os
import boto3
import bs4
import json

In [13]:
load_dotenv()
TEXT_MODEL = os.getenv("TEXT_MODEL")
SERVICE_NAME = os.getenv("AWS_SERVICE_NAME")
REGION_NAME = os.getenv("AWS_REGION_NAME")
ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY")
ACCESS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [14]:
client = boto3.client(service_name=SERVICE_NAME,
                       region_name = REGION_NAME,
                       aws_access_key_id=ACCESS_KEY_ID,
                       aws_secret_access_key=ACCESS_SECRET_KEY)

In [15]:
def generate_ai_text(max_tokens=256, temperature=1, top_p=0.8, top_k=50):
    try:
        prompt = "describe amazon"
        model_id = TEXT_MODEL
        body = {
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "top_k": top_k
        }

        response = client.invoke_model(
            modelId=model_id, 
            body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        outputs = response_body.get("outputs")
        generated_text = outputs[0]['text'].replace('"', '')
        generated_text = generated_text.split('\n', 1)[0].strip()
        
        return generated_text
    
    except Exception as e:
        return {"error": str(e)}

In [16]:
generate_ai_text()

{'error': 'An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.'}

In [1]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("cricbuzz.txt")
text_documents=loader.load()
text_documents


[Document(page_content="Rashid Khan, Naveen Ul Haq carry Afghanistan to World Cup semis\nby  Cricbuzz Staff  •  Last updated on Tue, Jun 25, 2024, 11:44 AM\nShare\nTweet\nRashid Khan picked a four-wicket haul in his team's win\nRashid Khan picked a four-wicket haul in his team's win © AFP\nTwo days after beating Australia, Afghanistan went one step further with an incredible win over Bangladesh to advance to the semifinal of the Men's T20 World Cup 2024. Once India beat Australia in the morning game, the requirement was clear: a win would take Afghanistan through ahead of the 2021 champions. However, Bangladesh came as a thorn in their flesh, putting in a commendable bowling performance to restrict Afghanistan to 115/5 in 20 overs. The conditions got a bit better with rain spells and Litton Das showed batting resolve but Rashid Khan (4 for 23) and Naveen Ul Haq (4 for 26) combined to take their side to the last-four.\n\nWhere the game was won:\n\nBangladesh would've comfortably gotten 

In [1]:
import requests
import xml.etree.ElementTree as ET

# URL of the RSS feed
url = "https://feeds.feedburner.com/ign/games-all"

# Fetch the content of the RSS feed
response = requests.get(url)
response.raise_for_status()  # Raises an HTTPError for bad responses

# Parse the XML content of the feed
root = ET.fromstring(response.content)

# Iterate through each item in the RSS feed
for item in root.findall('.//item'):
    title = item.find('title').text
    description = item.find('description').text
    image_url = item.find('.//media:content', namespaces={'media': 'http://search.yahoo.com/mrss/'}).attrib['url'] if item.find('.//media:content', namespaces={'media': 'http://search.yahoo.com/mrss/'}) is not None else 'No image available'
    print(f'Title: {title}')
    print(f'Description: {description}')
    print(f'Image URL: {image_url}')
    print('---')

Title: Elden Ring: Shadow of the Erdtree Gets First Patch in Update 1.12.2
Description: Elden Ring developer FromSoftware has released its first patch for the game following the release of Shadow of the Erdtree, bringing it up to Update 1.12.2.
Image URL: https://assets-prd.ignimgs.com/2024/02/26/elden-ring-dlc-sale-ign-2-1708957363507.png
---
Title: Save 50% Off Unicorn Overlord for Nintendo Switch
Description: The best console strategy game in recent history
Image URL: https://assets-prd.ignimgs.com/2024/06/25/unicornoverlord3-1719343037851.jpg
---
Title: Gayming Awards 2024 Winners: Baldur's Gate 3 Wins Over Tears of the Kingdom and Spider-Man 2
Description: The 2024 Gayming Awards winners have been revealed, with Larian Studios and its 2023 RPG Baldur's Gate 3, winning the most awards, including Game of the Year.
Image URL: https://assets-prd.ignimgs.com/2024/06/25/baldursgate3-review-blogroll-1692393747093-1719327576662.jpg
---
Title: Luigi's Mansion 2 HD Developer Revealed to be 

In [15]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://www.cricbuzz.com/cricket-news/130856/rashid-khan-naveen-ul-haq-carry-afghanistan-to-world-cup-semis",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("nws-dtl-hdln","cb-nws-dtl-itms")

                     )))

text_documents=loader.load()

In [16]:
text_documents

[Document(page_content=" Rashid Khan, Naveen Ul Haq carry Afghanistan to World Cup semis   Two days after beating Australia, Afghanistan went one step further with an incredible win over Bangladesh to advance to the semifinal of the Men's T20 World Cup 2024. Once India beat Australia in the morning game, the requirement was clear: a win would take Afghanistan through ahead of the 2021 champions. However, Bangladesh came as a thorn in their flesh, putting in a commendable bowling performance to restrict Afghanistan to 115/5 in 20 overs. The conditions got a bit better with rain spells and Litton Das showed batting resolve but Rashid Khan (4 for 23) and Naveen Ul Haq (4 for 26) combined to take their side to the last-four.    Where the game was won:    Bangladesh would've comfortably gotten over the line if not for the collapse in the middle-overs. Contrastingly, Afghanistan did well to lose only one wicket in this phase even as they were in need of a move on from a slow start.      The 

In [ ]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content="Rashid Khan, Naveen Ul Haq carry Afghanistan to World Cup semis   Two days after beating Australia, Afghanistan went one step further with an incredible win over Bangladesh to advance to the semifinal of the Men's T20 World Cup 2024. Once India beat Australia in the morning game, the requirement was clear: a win would take Afghanistan through ahead of the 2021 champions. However, Bangladesh came as a thorn in their flesh, putting in a commendable bowling performance to restrict Afghanistan to 115/5 in 20 overs. The conditions got a bit better with rain spells and Litton Das showed batting resolve but Rashid Khan (4 for 23) and Naveen Ul Haq (4 for 26) combined to take their side to the last-four.    Where the game was won:    Bangladesh would've comfortably gotten over the line if not for the collapse in the middle-overs. Contrastingly, Afghanistan did well to lose only one wicket in this phase even as they were in need of a move on from a slow start.      The m

In [19]:
## Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [22]:
query = "Who WON THE MATCH"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

[Document(page_content="Rashid Khan, Naveen Ul Haq carry Afghanistan to World Cup semis   Two days after beating Australia, Afghanistan went one step further with an incredible win over Bangladesh to advance to the semifinal of the Men's T20 World Cup 2024. Once India beat Australia in the morning game, the requirement was clear: a win would take Afghanistan through ahead of the 2021 champions. However, Bangladesh came as a thorn in their flesh, putting in a commendable bowling performance to restrict Afghanistan to 115/5 in 20 overs. The conditions got a bit better with rain spells and Litton Das showed batting resolve but Rashid Khan (4 for 23) and Naveen Ul Haq (4 for 26) combined to take their side to the last-four.    Where the game was won:    Bangladesh would've comfortably gotten over the line if not for the collapse in the middle-overs. Contrastingly, Afghanistan did well to lose only one wicket in this phase even as they were in need of a move on from a slow start.      The m

In [ ]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents[:15], OpenAIEmbeddings())

In [ ]:
!pip install llama-index

# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = ""


import logging
import sys
import requests

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

# download paul graham's essay
response = requests.get("https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1")
essay_txt = response.text
with open("pg_essay.txt", "w") as fp:
  fp.write(essay_txt)
  
  
  # load documents
documents = SimpleDirectoryReader(input_files=['pg_essay.txt']).load_data()


index = VectorStoreIndex.from_documents(documents)


# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(similarity_top_k=2)

response = query_engine.query(
    "What did the author do growing up?",
)


print(response.source_nodes[0].node.get_text())

In [ ]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=api_key, top_n=2) # return top 2 nodes from reranker

query_engine = index.as_query_engine(
    similarity_top_k=10, # we can set a high top_k here to ensure maximum relevant retrieval
    node_postprocessors=[cohere_rerank], # pass the reranker to node_postprocessors
)

response = query_engine.query(
    "What did Elon Musk do?",
)

In [3]:
import requests
import xml.etree.ElementTree as ET

# URL of the RSS feed
url = "https://www.aluxurytravelblog.com/feed/"

# Headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Fetch the content of the RSS feed
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raises an HTTPError for bad responses

# Parse the XML content of the feed
root = ET.fromstring(response.content)

# Iterate through each item in the RSS feed
for item in root.findall('.//item'):
    title = item.find('title').text
    description = item.find('description').text
    link = item.find('link').text
    pub_date = item.find('pubDate').text
    creator = item.find('{http://purl.org/dc/elements/1.1/}creator').text

    print(f'Title: {title}')
    print(f'Description: {description}')
    print(f'Link: {link}')
    print(f'Publication Date: {pub_date}')
    print(f'Creator: {creator}')
    print('---')


Title: Luxury living with Raffles in Cambodia
Description: <p>For discerning travellers visiting Cambodia who are looking for a luxury living experience, an experience full of unforgettable moments...</p>
<p>The post <a href="https://www.aluxurytravelblog.com/2024/06/25/luxury-living-with-raffles-in-cambodia/">Luxury living with Raffles in Cambodia</a> appeared first on <a href="https://www.aluxurytravelblog.com">A Luxury Travel Blog</a>.</p>

Link: https://www.aluxurytravelblog.com/2024/06/25/luxury-living-with-raffles-in-cambodia/
Publication Date: Tue, 25 Jun 2024 11:07:00 +0000
Creator: Paul Eyers
---
Title: Why Botswana is the perfect luxury safari experience
Description: <p>A diversity of wildlife, landscapes and luxury lodges is what makes Botswana such a special destination. Also, it is...</p>
<p>The post <a href="https://www.aluxurytravelblog.com/2024/06/24/botswana-luxury-safari-experience/">Why Botswana is the perfect luxury safari experience</a> appeared first on <a href="h

In [ ]:
content-wrap